# __Camino de Santiago__ 
## Pilgrimage Data Retrieveal from the Cathedral RESTful Server 
### Notebook uses async calls to avoid DoS defensive timeouts by the Server



#### __Step 1:__ Load libraries

In [110]:

import time            ### To work with time objects
import json            ### To Process returned JSON data
import pandas as pd    ### To work with dataframes
import asyncio         ### To make async request
import httpx           ### To make request to Cathedral URL
import datetime        ### To work with datetime
import openpyxl        ### To same to Excel files


#### __Step 2:__ Initialize Panda dataframes to be filled

In [111]:
############## Dataframes to fill
 
tablelist = [
    'Anho',
    'Mes',
    'Identificador',
    'TotalRegistros',
    'Autonomias',
    'Caminos',
    'Continentes',
    'Edades',
    'Grupos',
    'Medios',
    'Motivos',
    'Paises',
    'Procedencias',
    'Sexos'
]
totals_df = pd.DataFrame(
    columns=['Anho', 'Mes', 'Identificador', 'TotalRegistros'])
autonomous_coms_df = pd.DataFrame(
    columns=['id', 'Anho', 'Mes', 'Nombre', 'Total', 'Porcentaje'])
routes_df = pd.DataFrame(
    columns=['id', 'Anho', 'Mes', 'Nombre', 'Total', 'Porcentaje'])
continents_df = pd.DataFrame(
    columns=['id', 'Anho', 'Mes', 'Nombre', 'Total', 'Porcentaje'])
ages_df = pd.DataFrame(
    columns=['id', 'Anho', 'Mes', 'Nombre', 'Total', 'Porcentaje'])
groups_df = pd.DataFrame(
    columns=['id', 'Anho', 'Mes', 'Nombre', 'Total', 'Porcentaje'])
transportation_df = pd.DataFrame(
    columns=['id', 'Anho', 'Mes', 'Nombre', 'Total', 'Porcentaje'])
motives_df = pd.DataFrame(
    columns=['id', 'Anho', 'Mes', 'Nombre', 'Total', 'Porcentaje'])
countries_df = pd.DataFrame(
    columns=['id', 'Anho', 'Mes', 'Nombre', 'Total', 'Porcentaje'])
origin_df = pd.DataFrame(
    columns=['id', 'Anho', 'Mes', 'Nombre', 'Total', 'Porcentaje'])
gender_df = pd.DataFrame(
    columns=['id', 'Anho', 'Mes', 'Nombre', 'Total', 'Porcentaje'])


#### __Step 3:__ Define functions that calculate the valid range of dates that may be retrieved from the server

In [112]:
############# Function that calculates the range of valid months
def range_of_months(start_date, end_date):
    months = []

    for i in range(start_date.year * 12 + start_date.month, end_date.year*12+end_date.month):
        #months.append(datetime.date((i-13) // 12 + 1, (i-1) % 12 + 1, 1))
        date_iter = datetime.date((i-13) // 12 + 1, (i-1) % 12 + 1, 1)
        year = date_iter.year
        month = date_iter.month
        months.append([year, month])
    return months


data_start_date = datetime.date(2004, 1, 1) #start date is January 2004
data_end_date = datetime.datetime.now()  # datetime.date(2011, 2, 1)
year_months = []
year_months = range_of_months(data_start_date, data_end_date)
#print(year_months)

#### __Step 4:__ Define functions that concatenate newly retrieved data to previously retrieved data

In [113]:
def add_prefix(idx, yr, mnth, df):
    df.insert(0, "id", idx)
    df.insert(1, "Anho", yr)
    df.insert(2, "Mes", mnth)
    return df


def add_to_dataframe(df_running_totals, yr, mnth, df_to_add):
    #print('dataframe1:',df1)
    #print('dataframe2:', df2)
    idx = str(yr)+str(mnth).zfill(2)
    add_prefix(idx, yr, mnth, df_to_add)
    frames = [df_running_totals, df_to_add]
    tempdf = pd.concat(frames, ignore_index=True)
    #print('temp',tempdf)

    return tempdf




#### __Step 5:__ Defines function that splits and converts retrieved json data to various the dataframes

In [114]:
############# Async function that requests data from the Cathedral's server ################
############# waits for response from server before it continues with requests to avoid empty responses (a "denial of service" defense mechanism? )


async def get_data(mes, anho):
    eMes = str(mes)
    eAnho = str(anho)
    url = 'https://catedral.df-server.info/ws/wsCatedral.asmx/ObtenerEstadisticasMes?eAnho=' + eAnho + '&eMes=' + eMes
    # print("mes:",mes,"anho:",anho,"url:",url)
    client = httpx.AsyncClient()
    async with client.stream('GET', url) as response:
        async for chunk in response.aiter_text():
           # print("Chunk:",chunk)
            if len(chunk) > 0:
                json_data_chunk = json.loads(chunk)
                totals_row = [anho, mes, json_data_chunk['Identificador'],json_data_chunk['TotalRegistros']]
                global totals_df
                totals_df.loc[len(totals_df.index)] = totals_row

                # Concatenate new results and running total dataframes, split JSON by key
                global autonomous_coms_df
                autonomous_coms_df=add_to_dataframe(autonomous_coms_df, anho, mes, pd.DataFrame(json_data_chunk['Autonomias']))
                global routes_df
                routes_df=add_to_dataframe(routes_df, anho, mes, pd.DataFrame(json_data_chunk['Caminos']))
                global continents_df
                continents_df=add_to_dataframe(continents_df, anho, mes, pd.DataFrame(json_data_chunk['Continentes']))
                global ages_df
                ages_df=add_to_dataframe(ages_df, anho, mes, pd.DataFrame(json_data_chunk['Edades']))
                global groups_df
                groups_df=add_to_dataframe(groups_df, anho, mes, pd.DataFrame(json_data_chunk['Grupos']))
                global transportation_df
                transportation_df=add_to_dataframe(transportation_df, anho, mes, pd.DataFrame(json_data_chunk['Medios']))
                global motives_df
                motives_df=add_to_dataframe(motives_df, anho, mes, pd.DataFrame(json_data_chunk['Motivos']))
                global countries_df
                countries_df=add_to_dataframe(countries_df, anho, mes, pd.DataFrame(json_data_chunk['Paises']))
                global origin_df
                origin_df=add_to_dataframe(origin_df, anho, mes, pd.DataFrame(json_data_chunk['Procedencias']))
                global gender_df
                gender_df=add_to_dataframe(gender_df, anho, mes, pd.DataFrame(json_data_chunk['Sexos']))

                print('Mes:', eMes, 'Anho:', eAnho)
                #print("start:", json_data_chunk, ":end", len(chunk))
            else:
                break
    #async with httpx.AsyncClient() as client:
    #    response = await client.stream('GET',url)
    #    print(response)

    ######### TEST ###############
    """ 
    for yAnho in range(2004,2005):
        for xMes in range(1,12):
            await get_data(xMes,yAnho,)
    """    
    ######### TEST END ###########  


data_start_date = datetime.date(2004, 1, 1)
#data_end_date = datetime.date(2004, 3, 1)
data_end_date = datetime.datetime.now()  # datetime.date(2011, 2, 1)
year_months = []
year_months = range_of_months(data_start_date, data_end_date)
#print(year_months)


for year_month in range(len(year_months)):
    await get_data(year_months[year_month][1],year_months[year_month][0])

  

Mes: 1 Anho: 2004
Mes: 2 Anho: 2004
Mes: 3 Anho: 2004
Mes: 4 Anho: 2004
Mes: 5 Anho: 2004
Mes: 6 Anho: 2004
Mes: 7 Anho: 2004
Mes: 8 Anho: 2004
Mes: 9 Anho: 2004
Mes: 10 Anho: 2004
Mes: 11 Anho: 2004
Mes: 12 Anho: 2004
Mes: 1 Anho: 2005
Mes: 2 Anho: 2005
Mes: 3 Anho: 2005
Mes: 4 Anho: 2005
Mes: 5 Anho: 2005
Mes: 6 Anho: 2005
Mes: 7 Anho: 2005
Mes: 8 Anho: 2005
Mes: 9 Anho: 2005
Mes: 10 Anho: 2005
Mes: 11 Anho: 2005
Mes: 12 Anho: 2005
Mes: 1 Anho: 2006
Mes: 2 Anho: 2006
Mes: 3 Anho: 2006
Mes: 4 Anho: 2006
Mes: 5 Anho: 2006
Mes: 6 Anho: 2006
Mes: 7 Anho: 2006
Mes: 8 Anho: 2006
Mes: 9 Anho: 2006
Mes: 10 Anho: 2006
Mes: 11 Anho: 2006
Mes: 12 Anho: 2006
Mes: 1 Anho: 2007
Mes: 2 Anho: 2007
Mes: 3 Anho: 2007
Mes: 4 Anho: 2007
Mes: 5 Anho: 2007
Mes: 6 Anho: 2007
Mes: 7 Anho: 2007
Mes: 8 Anho: 2007
Mes: 9 Anho: 2007
Mes: 10 Anho: 2007
Mes: 11 Anho: 2007
Mes: 12 Anho: 2007
Mes: 1 Anho: 2008
Mes: 2 Anho: 2008
Mes: 3 Anho: 2008
Mes: 4 Anho: 2008
Mes: 5 Anho: 2008
Mes: 6 Anho: 2008
Mes: 7 Anho: 200

#### __Step 6:__ Rename columns to proper case

In [115]:
column_names = {
    "id": "ID",  "anho": "Anho", "mes": "Mes", "Nombre": "Descripcion", "Total": "Total",  "Porcentaje": "Porcentaje"}
autonomous_coms_df.rename(columns=column_names, inplace=True)

routes_df.rename(columns=column_names, inplace=True)
continents_df.rename(columns=column_names, inplace=True)
ages_df.rename(columns=column_names, inplace=True)
groups_df.rename(columns=column_names, inplace=True)
transportation_df.rename(columns=column_names, inplace=True)
motives_df.rename(columns=column_names, inplace=True)
countries_df.rename(columns=column_names, inplace=True)
origin_df.rename(columns=column_names, inplace=True)
gender_df.rename(columns=column_names, inplace=True)


#### __Step 7:__ Totals has an error, use genders to generate a true total using Pandas groupby with the sum() function

In [116]:
totals_true_df=gender_df.groupby(['ID','Anho','Mes'], as_index =False)['Total'].sum()


#### __Step 8:__ Converts tables format from __"long"__ to __"wide"__, in this step we also convert nulls to zero's

In [119]:

autonomous_coms_df2=autonomous_coms_df.groupby(['ID','Anho','Mes','Descripcion'], as_index =False)['Total'].sum()
autonomous_coms_wide_df=pd.pivot(autonomous_coms_df2,index=['ID','Anho','Mes'],columns='Descripcion',values='Total')
cols = autonomous_coms_df['Descripcion'].unique()
autonomous_coms_wide_df=autonomous_coms_wide_df[cols]
autonomous_coms_wide_df.fillna(0,inplace=True)
autonomous_coms_wide_df['Total']=autonomous_coms_wide_df.sum(axis=1)
autonomous_coms_wide_df

routes_df2=routes_df.groupby(['ID','Anho','Mes','Descripcion'], as_index =False)['Total'].sum()
routes_wide_df=pd.pivot(routes_df2,index=['ID','Anho','Mes'],columns='Descripcion',values='Total')
cols = routes_df['Descripcion'].unique()
routes_wide_df=routes_wide_df[cols]
routes_wide_df.fillna(0,inplace=True)
routes_wide_df['Total']=routes_wide_df.sum(axis=1)
routes_wide_df

continents_df2=continents_df.groupby(['ID','Anho','Mes','Descripcion'], as_index =False)['Total'].sum()
continents_wide_df=pd.pivot(continents_df2,index=['ID','Anho','Mes'],columns='Descripcion',values='Total')
cols = continents_df['Descripcion'].unique()
continents_wide_df=continents_wide_df[cols]
continents_wide_df.fillna(0,inplace=True)
continents_wide_df['Total']=continents_wide_df.sum(axis=1)
continents_wide_df


#ages_wide_df=pd.pivot(ages_df,index=['ID','Anho','Mes'],columns='Descripcion',values='Total')
#cols = ages_df['Descripcion'].unique()
#ages_wide_df=ages_wide_df[cols]
#ages_wide_df.fillna(0,inplace=True)
#ages_wide_df['Total']=ages_wide_df.sum(axis=1)
#ages_wide_df

ages_df2=ages_df.groupby(['ID','Anho','Mes','Descripcion'], as_index =False)['Total'].sum()
ages_wide_df=pd.pivot(ages_df2,index=['ID','Anho','Mes'],columns='Descripcion',values='Total')
cols = ages_df['Descripcion'].unique()

ages_wide_df=ages_wide_df[cols]
ages_wide_df.fillna(0,inplace=True)
ages_wide_df['Total']=ages_wide_df.sum(axis=1)

ages_wide_df


groups_df2=groups_df.groupby(['ID','Anho','Mes','Descripcion'], as_index =False)['Total'].sum()
groups_wide_df=pd.pivot(groups_df2,index=['ID','Anho','Mes'],columns='Descripcion',values='Total')
cols = groups_df['Descripcion'].unique()
groups_wide_df=groups_wide_df[cols]
groups_wide_df.fillna(0,inplace=True)
groups_wide_df['Total']=groups_wide_df.sum(axis=1)
groups_wide_df


transportation_df2=transportation_df.groupby(['ID','Anho','Mes','Descripcion'], as_index =False)['Total'].sum()
transportation_wide_df=pd.pivot(transportation_df2,index=['ID','Anho','Mes'],columns='Descripcion',values='Total')
cols = transportation_df['Descripcion'].unique()
transportation_wide_df=transportation_wide_df[cols]
transportation_wide_df.fillna(0,inplace=True)
transportation_wide_df['Total']=transportation_wide_df.sum(axis=1)
transportation_wide_df

motives_df2=motives_df.groupby(['ID','Anho','Mes','Descripcion'], as_index =False)['Total'].sum()
motives_wide_df=pd.pivot(motives_df2,index=['ID','Anho','Mes'],columns='Descripcion',values='Total')
cols = motives_df['Descripcion'].unique()
motives_wide_df=motives_wide_df[cols]
motives_wide_df.fillna(0,inplace=True)
motives_wide_df['Total']=motives_wide_df.sum(axis=1)
motives_wide_df

countries_df2=countries_df.groupby(['ID','Anho','Mes','Descripcion'], as_index =False)['Total'].sum()
countries_wide_df=pd.pivot(countries_df2,index=['ID','Anho','Mes'],columns='Descripcion',values='Total')
cols = countries_df['Descripcion'].unique()
countries_wide_df=countries_wide_df[cols]
countries_wide_df.fillna(0,inplace=True)
countries_wide_df['Total']=countries_wide_df.sum(axis=1)
countries_wide_df

origin_df2=origin_df.groupby(['ID','Anho','Mes','Descripcion'], as_index =False)['Total'].sum()
origin_wide_df=pd.pivot(origin_df2,index=['ID','Anho','Mes'],columns='Descripcion',values='Total')
cols = origin_df['Descripcion'].unique()
origin_wide_df=origin_wide_df[cols]
origin_wide_df.fillna(0,inplace=True)
origin_wide_df['Total']=origin_wide_df.sum(axis=1)
origin_wide_df

gender_df2=gender_df.groupby(['ID','Anho','Mes','Descripcion'], as_index =False)['Total'].sum()
gender_wide_df=pd.pivot(gender_df2,index=['ID','Anho','Mes'],columns='Descripcion',values='Total')
cols = gender_df['Descripcion'].unique()
gender_wide_df=gender_wide_df[cols]
gender_wide_df.fillna(0,inplace=True)
gender_wide_df['Total']=gender_wide_df.sum(axis=1)
gender_wide_df 



,,Descripcion,Hombre,Mujer,Desconocido,Sin establecer,Total
ID,Anho,Mes,,,,,
200401,2004,1,454.0,189.0,0.0,0.0,643.0
200402,2004,2,910.0,491.0,0.0,0.0,1401.0
200403,2004,3,1838.0,1267.0,0.0,0.0,3105.0
200404,2004,4,8437.0,7117.0,0.0,0.0,15554.0
200405,2004,5,9778.0,7082.0,0.0,0.0,16860.0
...,...,...,...,...,...,...,...
202206,2022,6,28175.0,29837.0,0.0,0.0,58012.0
202207,2022,7,31151.0,36223.0,0.0,0.0,67374.0
202208,2022,8,40301.0,45541.0,0.0,0.0,85842.0


#### __Step 8:__ Output Excel files

In [120]:
### Path to use to save the dataframes to Excel files  ###
save_path_xl="C:/Sites/CaminoAwait/Data/Excel/"

totals_df.to_excel(save_path_xl+"camino_totals.xlsx", engine="openpyxl")
autonomous_coms_wide_df.to_excel(save_path_xl+"camino_autonomous_comm.xlsx", engine="openpyxl")
routes_wide_df.to_excel(save_path_xl+"camino_routes.xlsx", engine="openpyxl")
continents_wide_df.to_excel(save_path_xl+"camino_continents.xlsx", engine="openpyxl")
ages_wide_df.to_excel(save_path_xl+"camino_ages.xlsx", engine="openpyxl")
groups_wide_df.to_excel(save_path_xl+"camino_groups.xlsx", engine="openpyxl")
transportation_wide_df.to_excel(save_path_xl+"camino_transportation.xlsx", engine="openpyxl")
motives_wide_df.to_excel(save_path_xl+"camino_motives.xlsx", engine="openpyxl")
countries_wide_df.to_excel(save_path_xl+"camino_countries.xlsx", engine="openpyxl")
origin_wide_df.to_excel(save_path_xl+"camino_origin.xlsx", engine="openpyxl")
gender_wide_df.to_excel(save_path_xl+"camino_gender.xlsx", engine="openpyxl")
totals_true_df.to_excel(save_path_xl+"camino_totals_true.xlsx", engine="openpyxl")




#### __Step 9:__ Output CSV files

In [121]:
### Path to use to save the dataframes to CSV files  ###
save_path_csv="C:/Sites/CaminoAwait/Data/"

totals_df.to_csv(save_path_csv+"camino_totals.csv",  header=True, index=True)
autonomous_coms_wide_df.to_csv(save_path_csv+"camino_autonomous_comm.csv",  header=True, index=True)
routes_wide_df.to_csv(save_path_csv+"camino_routes.csv",  header=True, index=True)
continents_wide_df.to_csv(save_path_csv+"camino_continents.csv",  header=True, index=True)
ages_wide_df.to_csv(save_path_csv+"camino_ages.csv",  header=True, index=True)
groups_wide_df.to_csv(save_path_csv+"camino_groups.csv",  header=True, index=True)
transportation_wide_df.to_csv(save_path_csv+"camino_transportation.csv",  header=True, index=True)
motives_wide_df.to_csv(save_path_csv+"camino_motives.csv",  header=True, index=True)
countries_wide_df.to_csv(save_path_csv+"camino_countries.csv",  header=True, index=True)
origin_wide_df.to_csv(save_path_csv+"camino_origin.csv",  header=True, index=True)
gender_wide_df.to_csv(save_path_csv+"camino_gender.csv",  header=True, index=True)
totals_true_df.to_csv(save_path_csv+"camino_totals_true.csv",  header=True, index=True)


#### __END__